In [ ]:
import os
import torch
import logging
import argparse
import math
import numpy as np
from tqdm import tqdm
import multiprocessing
import time
from utils import convert_examples_to_features, read_review_examples
import json
from torch.utils.data import Dataset
from itertools import cycle
from torch.utils.data import DataLoader, RandomSampler
from torch.utils.data.distributed import DistributedSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from models import build_or_load_gen_model
from configs import add_args, set_seed, set_dist
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
import os
from transformers import T5Tokenizer, T5Model
from utils import read_review_examples, convert_examples_to_features, TextDataset
from models import build_or_load_gen_model
from run_pre_training import get_loaders
import multiprocessing
import torch.distributed as dist

In [ ]:
class FakeArgs(object):
    def __init__(self):
        self.max_source_length = 512
        self.max_target_length = 256
        self.model_type = "t5"
        self.config_name = "t5-base"
        self.add_lang_ids = True
        self.tokenizer_path = "t5-base"
        self.model_name_or_path = "t5-base"
        self.load_model_path = None
        self.train_path = "../../../lzzz/processed"
        self.train_batch_size = 1
        self.langs = ["ruby"]
        self.cpu_count = 3
        self.num_train_epochs = 2
        self.gradient_accumulation_steps = 1
        self.mask_rate = 0.15
        self.from_scratch = True

args = FakeArgs()
config, model, tokenizer = build_or_load_gen_model(args)
pool = multiprocessing.Pool(args.cpu_count)
packs, steps = get_loaders(args, tokenizer, pool)
dataset, _, loader = packs[0]
data = next(loader)

In [ ]:
model.config

In [ ]:
ids = tokenizer.encode("testo")
tokenizer.convert_ids_to_tokens(ids)

In [ ]:
inputs = data[0].source_ids
labels = data[0].source_labels
targets = data[0].target_ids
# print("".join())

In [ ]:
def prettyprint(inputs):
    for i, tok in enumerate(inputs):
        if i > 0 and str(tok).startswith("<e"):
            print("")
        print(str(tok) + " ", end="")
toks = tokenizer.convert_ids_to_tokens(inputs)
labs = labels
tgts = tokenizer.convert_ids_to_tokens(targets)
# print(toks)
prettyprint(toks)
print("\n\n\n\n\n")
prettyprint(labs)
print("\n\n\n\n\n")
prettyprint(tgts)
print("\n\n\n\n\n")


In [ ]:
print("\n".join(dataset.examples[0].prevlines))
print("\n\n\n\n\n")
print("\n".join(dataset.examples[0].lines))
print("\n\n\n\n\n")
print("\n".join(dataset.examples[0].afterlines))
print("\n\n\n\n\n")
print(dataset.examples[0].msg)

In [33]:
import json
data = []
idx = 0
with open("../../../lzzz/processed/ruby_cls.jsonl", "r") as f:
    for line in f:
        js = json.loads(line)
        data.append(js)
        # print(js["patch"])
        idx += 1
        if idx == 100:
            break

@@ -47,28 +47,28 @@
 			end
 
 			def create
-				instance_variable_set("@#{resource_model_name}", resource_model.new(params[resource_model_name.to_sym]))
+				instance_variable_set("@#{resource_handler.model_name}", resource_handler.model.new(params[resource_handler.namespaced_model_name.to_sym]))
 				resource_instance_variable.save
 				render_errors_or_redirect(
-					resource_instance_variable,
-					resource_url_scope.url_for({:action => :index}),
-					flash_notice_for_resource_action
+						resource_instance_variable,
+						resources_path,
+						flash_notice_for_resource_action
 				)
 			end
 
 			def update
-				resource_instance_variable.update_attributes(params[resource_model_name.to_sym])
+				resource_instance_variable.update_attributes(params[resource_handler.namespaced_model_name.to_sym])
 				render_errors_or_redirect(
-					resource_instance_variable,
-					resource_url_scope.url_for({:action => :index}),
-					flash_notice_for_resource_action
+						resource_instan